Module Imports

In [13]:
import os
import re
import csv
import numpy as np
import pandas as pd
import string
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_files
from sklearn.metrics import accuracy_score,classification_report
from sklearn.linear_model import LogisticRegression
from string import digits


Data imports and parsing

In [14]:
data_dir = '/content/drive/MyDrive/10blogs'#a dir of 10 blogs in xml
data_xml=[]
data_txt={}

In [15]:
#content words
content_words=[]
with open('/content/drive/MyDrive/research/content_words.txt') as f: #content_words txt file
    #Content_list is the list that contains the read lines.     
    for line in f:
      content_words.append(line.translate({ord(c): None for c in string.whitespace}))
    cont = []
    for i in range(len(content_words)):
      remove_digits = str.maketrans('', '', digits)
      cont.append(content_words[i].translate(remove_digits))
    
    


In [16]:
for filename in os.listdir(data_dir):
    f = os.path.join(data_dir,filename)
    if os.path.isfile(f):
          data_xml.append(f)

In [17]:
for i in data_xml:
  tree = ET.parse(i)  #parsing
  root = tree.getroot()   #parsing
  textArr = []
  for child in root:
      if child.tag=='post':
        textArr.append(str(child.text))
        data_txt[i]=textArr
      


In [18]:
texts = list(data_txt.values())
author = list(data_txt.keys())

for i in range(len(texts)):
  texts[i] = str(texts[i])

Limiting number of entries and characters inside each blogs


In [19]:
for i in range(len(texts)):
  string1= texts[i][:len(texts[i])//2]
  string2= texts[i][len(texts[i])//2:]
  texts[i] = [string1[:1500], string2[:1500]]

In [20]:
fixed_author = []
for i in range(10):
  fixed_author.append(author[i].strip('/content/drive/MyDrive/10blogs/'))
  

In [21]:
for i in range(len(texts)):

  print(len(texts[i]))

2
2
2
2
2
2
2
2
2
2


Creating a data frame

In [22]:
df = pd.DataFrame({'texts':texts,
                   'author':fixed_author})
df=df.explode('texts')

In [23]:
xfeature = df['texts']
ylabel = df['author']

In [24]:
# list of documents
docs = xfeature
# create the transform
vectorizer = CountVectorizer(vocabulary=cont)
# tokenize and build vocab
X = vectorizer.fit_transform(docs)
print('vocabulary: ', vectorizer.vocabulary_)
# summarize encoded vector
print('shape: ', X.shape)
print('vectors: ', X.toarray())

vocabulary:  {'a': 0, 'about': 1, 'above': 2, 'across': 3, 'after': 4, 'afterwards': 5, 'again': 6, 'against': 7, 'all': 8, 'almost': 9, 'alone': 10, 'along': 11, 'already': 12, 'also': 13, 'although': 14, 'always': 15, 'am': 16, 'among': 17, 'amongst': 18, 'amoungst': 19, 'an': 20, 'and': 21, 'another': 22, 'any': 23, 'anyhow': 24, 'anyone': 25, 'anything': 26, 'anyway': 27, 'anywhere': 28, 'are': 29, 'around': 30, 'as': 31, 'at': 32, 'be': 33, 'became': 34, 'because': 35, 'been': 36, 'before': 37, 'beforehand': 38, 'behind': 39, 'being': 40, 'below': 41, 'beside': 42, 'besides': 43, 'between': 44, 'beyond': 45, 'both': 46, 'but': 47, 'by': 48, 'can': 49, 'cannot': 50, 'could': 51, 'dare': 52, 'despite': 53, 'did': 54, 'do': 55, 'does': 56, 'done': 57, 'down': 58, 'during': 59, 'each': 60, 'eg': 61, 'either': 62, 'else': 63, 'elsewhere': 64, 'enough': 65, 'etc': 66, 'even': 67, 'ever': 68, 'every': 69, 'everyone': 70, 'everything': 71, 'everywhere': 72, 'except': 73, 'few': 74, 'first

In [25]:
X_train,X_test,y_train,y_test = train_test_split(X, ylabel, train_size=0.10, random_state=42)
                                            

In [26]:
X_train.shape

(2, 278)

Tests

In [27]:
clf = MultinomialNB() #classification
clf.fit(X_train , y_train)

MultinomialNB()

In [28]:
y_pred = clf.predict(X_test)
print(accuracy_score(y_test,clf.predict(X_test)))
print(classification_report(y_test,y_pred))

0.1111111111111111
                                          precision    recall  f1-score   support

     2000257.female.14.indUnk.Scorpio.xm       0.00      0.00      0.00         2
29253.female.26.HumanResources.Cancer.xm       0.00      0.00      0.00         2
          37626.male.24.indUnk.Gemini.xm       0.20      1.00      0.33         1
       40964.female.23.RealEstate.Leo.xm       0.08      1.00      0.14         1
      43781.female.25.Education.Aries.xm       0.00      0.00      0.00         2
 48638.male.36.Technology.Sagittarius.xm       0.00      0.00      0.00         2
          566970.male.27.indUnk.Libra.xm       0.00      0.00      0.00         2
   61176.male.33.Technology.Capricorn.xm       0.00      0.00      0.00         2
         67459.male.34.Arts.Capricorn.xm       0.00      0.00      0.00         2
             854.male.25.indUnk.Virgo.xm       0.00      0.00      0.00         2

                                accuracy                           0.11      

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [29]:
logit = LogisticRegression() #logistic regression
logit.fit(X_train,y_train)


LogisticRegression()

In [30]:
accuracy_score(y_test,logit.predict(X_test))

0.1111111111111111

In [31]:
f26hrcancer = ["Billy Zane  and Jaoquin Phoenix are half-brothers.  Their father may or may not be  the  Lazarus (he could just be  a  Lazarus- type ), this mystical paternity has granted them the ability to raise the recently deceased from the dead.    Do they have a mission?  Yes, to find out the fate of their father and, if he still lives, kill him and set him free from the bonds of the Earth because for every year the Dad is alive, a single soul must take his place in the afterlife.  A soul that would have otherwise continued to live.  Oh, the angst!  Are the brothers raising the right people from the dead? Or are they making things worse?  Would they be better off doing nothing, or not bringing people back and just killing Dad?  How do they find all of this out?  Does a little man who lives in a cave and wears a white robe with an equally white beard tell them?  No!  Their mothers belonged to a cult - both died in childbirth, of course - and they were raised by a woman who amounts to the cult's den mother (because even cults need someone to clean the bathrooms).    Is there a conflict between the brothers? Of course!  Jaoquin's character is afraid of what will be revealed and would rather turn back than continue on and Billy's character drags him forward by sheer inertia.    But who will play Dad?  That is the question.  "]
vect1 = vectorizer.transform(f26hrcancer).toarray()

m25virgo = ["For someone new to London (or just visiting as I have in the past) the London Underground is a saviour (despite being hot, crowded and late). Get a travelcard and you can just from one place to another without any problem at all.    But the problem with navigating London via the tube is that it gives a totally skewed impression of the Geography of the place. People get the tube when walking would have taken half the time, if only they knew where they were going. For example, in the past I've gone from Tottenham Court Road to Covent Garden by getting the Northern line to Leicester Square and then changing on to the Picadilly line. Now, that probably took about 25 minutes, but if I'd just walkd I could have done it in 15. Ish.         A couple of weeks ago I bought an A-Z which has put a few things into perspective, but for the most part I'm still stuck on tube-geography. Yesterday I came across a  urlLink handy little web page  which gives a number of tube maps, including a  urlLink geographically correct map of the underground .  Nice to see, but doesn't really give you an impression of how you can get from place to another.    But also included on that page is a PDF of the  urlLink geographical map superimprosed onto a street map ! It shows landmarks, train stations and railway lines, tube stop and lines, all in the correct locations, so now you really can see just how quick it would have been to walk instead of fighting the commute! "]
vect2 = vectorizer.transform(m25virgo).toarray()

m36techsagi =["Microsoft, it's long been known, has the digital music market (refined and defined by Apple) in its sights. Looks like they're getting close enough to the initiation of their plans that they're willing to start talking trash: "]
vect3 = vectorizer.transform(m36techsagi).toarray()


In [32]:
clf.predict(vect2)

array(['40964.female.23.RealEstate.Leo.xm'], dtype='<U33')

In [33]:
logit.predict(vect2)

array(['40964.female.23.RealEstate.Leo.xm'], dtype=object)